In [12]:
import numpy as np
from random import randint
from sklearn.preprocessing import MinMaxScaler

In [13]:
# data
train_labels = []   
train_samples = []
for i in range(1000):
    train_labels.append(0)
    train_samples.append(randint(13, 64))
    train_labels.append(1)
    train_samples.append(randint(65, 100))
for i in range(50):
    train_labels.append(1)
    train_samples.append(randint(13, 64))
    train_labels.append(0)
    train_samples.append(randint(65, 100))
    
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)

In [14]:
# preprocessing
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))

In [15]:
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

Using TensorFlow backend.


In [16]:
# build model
model = Sequential([
    Dense(16, input_shape=(1,), activation="relu"),
    Dense(32, activation="relu"),
    Dense(2, activation="softmax")
])

W0703 09:44:00.443696 12576 deprecation_wrapper.py:119] From D:\virtualenvs\congyuml\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0703 09:44:00.460652 12576 deprecation_wrapper.py:119] From D:\virtualenvs\congyuml\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0703 09:44:00.464642 12576 deprecation_wrapper.py:119] From D:\virtualenvs\congyuml\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                32        
_________________________________________________________________
dense_2 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________
